# 使用编码工具
编码工具工作流程示意
## 定义字典
文字是抽象概念，将抽象的文字数字化，让计算机可以进行数学运算
示意字典


In [7]:
vocab = {
  '<SOS>': 0,
  '<EOS>': 1,
  'quick': 2,
  'brown': 3,
  'fox': 4,
  'jumps': 5,
  'over': 6,
  'a': 7,
  'lazy': 8,
  'dog': 9,
  'the': 10,
}

## 句子预处理
在句子被分词前一般都会对句子进行简单的预处理。
方法：
- 长句子截成短句子
- 在句子中添加特殊符号

In [8]:
sent = 'the quick brown fox jumps over a lazy dog'
sent = '<SOS> ' + sent + ' <EOS>'
print(sent)

<SOS> the quick brown fox jumps over a lazy dog <EOS>


## 分词（英文）
将句子分成一个一个的单词，英文可以直接使用空格进行分词

In [9]:
words = sent.split()
print(words)

['<SOS>', 'the', 'quick', 'brown', 'fox', 'jumps', 'over', 'a', 'lazy', 'dog', '<EOS>']


## 编码
计算机是不认识单词的需要将单词映射到已经建立好的字典里对应的数字

In [13]:
for w in words:
  print("{} encode is: {}".format(w, vocab[w]))
print("encode is {}".format([vocab[i] for i in words ]))

<SOS> encode is: 0
the encode is: 10
quick encode is: 2
brown encode is: 3
fox encode is: 4
jumps encode is: 5
over encode is: 6
a encode is: 7
lazy encode is: 8
dog encode is: 9
<EOS> encode is: 1
encode is [0, 10, 2, 3, 4, 5, 6, 7, 8, 9, 1]


## 使用编码工具
hunggingface 提供的中文编码工具 bert-base-chinese
### 1. 加载编码工具
代码如下：

In [14]:
# !pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.1 MB/s eta 0:00:00


In [37]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(
    pretrained_model_name_or_path = 'bert-base-chinese',
    cache_dir=None,
)

###2. 准备实验数据
准备测试句子，测试编码工具。

In [32]:
sents = [
  '你站在桥上看风景',
  '看风景的人在楼上看你',
  '明月装饰了你的窗子',
  '你装饰了别人的梦',
]

### 3. 基本的编码函数


In [33]:
dir(tokenizer)
out = tokenizer.encode(
    text = sents[0],
    text_pair = None, # 另一个句子
    # truncation=True, 句子长度大于 max_length 时截断句子
    # padding='max_length', 句子长度小于 max_length 时填充句子
    add_special_tokens=True,
    # max_length=25,
)
print(out)
print(tokenizer.decode(out))

[101, 872, 4991, 1762, 3441, 677, 4692, 7599, 3250, 102]
[CLS] 你 站 在 桥 上 看 风 景 [SEP]


### 4.进阶编码函数

In [34]:
out = tokenizer.encode_plus(
    text = sents[0],
    text_pair = sents[1],
    add_special_tokens = True,
    # return token_type_ids
    return_token_type_ids = True,
    # return attention_mask
    return_attention_mask = True,
    # return special_tokens_mask
    return_special_tokens_mask = True,
    return_length = True,
)
print("length sents[0] + sents[1]: {}".format(len(sents[0])+len(sents[1])))
# print(out)
for k,v in out.items():
  print(k, ':', v)

tokenizer.decode(out['input_ids'])

length sents[0] + sents[1]: 18
input_ids : [101, 872, 4991, 1762, 3441, 677, 4692, 7599, 3250, 102, 4692, 7599, 3250, 4638, 782, 1762, 3517, 677, 4692, 872, 102]
token_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
special_tokens_mask : [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
length : 21


'[CLS] 你 站 在 桥 上 看 风 景 [SEP] 看 风 景 的 人 在 楼 上 看 你 [SEP]'

token_type_ids：表示那个时第一个句子，那个是第二个句子  
special_tokens_mask：标识句子中的特殊符号  
attention_mask: 标识那些位置是PAD，是为0，不是为1  
length: 编码后长度
### 5. 批量的编码函数
以上 encode 和 encode_plus 函数都是对一句话，或者一对句子进行编码，实际可能是一篇文章，上千句话，为了提高效率可以使用 batch_encode_plus() 函数批量处理句子代码如下：  

In [46]:
out = tokenizer.batch_encode_plus(
    batch_text_or_text_pairs = [(sents[0],sents[1]),(sents[2],sents[3])],
    # batch_text_or_text_pairs = [sents[0],sents[1],sents[2]], 多个句子
    # batch_text_or_text_pairs = sents, 多个句子直接使用 列表传参
    # 添加特殊字符
    add_special_tokens = True,
    # return token_type_ids
    return_token_type_ids = True,
    # return attention_mask
    return_attention_mask = True,
    # return special_tokens_mask
    return_special_tokens_mask = True,
    return_length = True,
    # return offsets_mapping, 标识每个词的起始位置
    return_offsets_mapping = True,
)
for k,v in out.items():
  print(k, ":", v)

tokenizer.decode(out['input_ids'][1])

input_ids : [[101, 872, 4991, 1762, 3441, 677, 4692, 7599, 3250, 102, 4692, 7599, 3250, 4638, 782, 1762, 3517, 677, 4692, 872, 102], [101, 3209, 3299, 6163, 7652, 749, 872, 4638, 4970, 2094, 102, 872, 6163, 7652, 749, 1166, 782, 4638, 3457, 102]]
token_type_ids : [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
attention_mask : [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]
special_tokens_mask : [[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
offset_mapping : [[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (0, 0)], [(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4,

'[CLS] 明 月 装 饰 了 你 的 窗 子 [SEP] 你 装 饰 了 别 人 的 梦 [SEP]'

### 6.对字典操作
查看字典

In [47]:
vocb = tokenizer.get_vocab()
type(vocb), len(vocb), '名誉' in vocb  # vocb 类型，vocb 长度，'名誉'词是否在 vocb 中

(dict, 21128, False)

添加新词、特殊符号到字典中

In [48]:
tokenizer.add_tokens(new_tokens=["明月","名誉",'窗子','装饰'])
tokenizer.add_special_tokens({'eos_token':'<EOS>'})

1

In [50]:
out = tokenizer.encode(
    text = '明月装饰了你的窗子<EOS>',
    add_special_tokens=True,
)
print(out)
tokenizer.decode(out)

[101, 21128, 21131, 749, 872, 4638, 21130, 21132, 102]


'[CLS] 明月 装饰 了 你 的 窗子 <EOS> [SEP]'